In [1]:
import os
import pandas as pd
from google.cloud import bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/ABCBankChurnRate/.config/sa_credentials.json"

In [3]:
client = bigquery.Client()
print(client.project)

kagglebigquerybankchurn


In [7]:
import os

# Manually load environment variables inside the notebook (if not set)
if not os.getenv("KAGGLE_USERNAME") or not os.getenv("KAGGLE_KEY"):
    os.environ["KAGGLE_USERNAME"] = "your_kaggle_username"
    os.environ["KAGGLE_KEY"] = "your_kaggle_api_key"

print("✅ Kaggle credentials are set in the notebook.")


✅ Kaggle credentials are set in the notebook.


In [9]:
from kaggle.api.kaggle_api_extended import KaggleApi
import glob

# Set Kaggle credentials from environment variables
if 'KAGGLE_USERNAME' not in os.environ or 'KAGGLE_KEY' not in os.environ:
	raise EnvironmentError("Kaggle credentials not found in environment variables")

# Initialize Kaggle API
api = KaggleApi()
api.authenticate()  # Uses the environment variable for authentication

# Define Kaggle dataset reference
dataset_name = "gauravtopre/bank-customer-churn-dataset"  # Replace with the actual Kaggle dataset
download_path = "/tmp"  # Temporary location, not inside your repo

# Download dataset (ZIP file)
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

# Find the downloaded CSV file
csv_files = glob.glob(f"{download_path}/*.csv")
assert len(csv_files) > 0, "No CSV files found. Check dataset name."
csv_file_path = csv_files[0]  # Assuming the dataset has a single CSV file

print(f"✅ Dataset downloaded: {csv_file_path}")

Dataset URL: https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset
✅ Dataset downloaded: /tmp/Bank Customer Churn Prediction.csv


In [10]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Define project, dataset, and table details
project_id = client.project  # Auto-fetch project ID
dataset_id = "churn_analysis"  # Choose a dataset name
table_id = "kaggle_churn"  # Choose a table name
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Define dataset (if it doesn’t exist)
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"  # Adjust region as needed
client.create_dataset(dataset, exists_ok=True)

# Define table schema (automatically detect types)
job_config = bigquery.LoadJobConfig(
    autodetect=True,  # Automatically infer column types
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip the header row
)

# Load data from CSV to BigQuery table
with open(csv_file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, full_table_id, job_config=job_config)

# Wait for job completion
job.result()

print(f"✅ Dataset uploaded to BigQuery: {full_table_id}")


✅ Dataset uploaded to BigQuery: kagglebigquerybankchurn.churn_analysis.kaggle_churn
